# Computational Analysis of Sound and Music

# MIR 3 - Music Transcription - 1

Dr.-Ing. Jakob Abeßer, jakob.abesser@idmt.fraunhofer.de

**Last update:** 12.05.2024

**Outline**

In this notebook, you will learn how to implement a simple **multipitch estimation** algorithm using a **U-Net deep neural network**.

In [ ]:
!pip install wget

In [ ]:
import glob
import os
import librosa
import numpy as np
import matplotlib.pyplot as pl
import numpy as np
import IPython.display as ipd
import wget
import zipfile
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPool2D, concatenate, UpSampling2D

## Dataset

We need a small dataset of piano recordings with corresponding multipitch annotations.

We use 4 files of the **SMD Dataset** (https://www.audiolabs-erlangen.de/resources/MIR/SMD/midi):
- Bach_BWV888-02_008_20110315-SMD
- Beethoven_Op027No1-02_003_20090916-SMD
- Chopin_Op028-01_003_20100611-SMD
- Chopin_Op028-03_003_20100611-SMD


In [ ]:
if not os.path.isfile('SMD.zip'):
    print('Please wait a couple of seconds ...')
    wget.download('https://github.com/machinelistening/machinelistening.github.io/blob/master/SMD.zip?raw=true', 
                      out='SMD.zip', bar=None)
    print('SMD downloaded successfully ...')
else:
    print('Files already exist!')
    
if not os.path.isdir('SMD.zip'):
    print("Let's unzip the file ... ")
    assert os.path.isfile('SMD.zip')
    with zipfile.ZipFile('SMD.zip', 'r') as f:
        # Entpacke alle Inhalte in das angegebene Verzeichnis
        f.extractall('.')
    assert os.path.isdir('SMD')
    print("All done :)")

dir_dataset = 'SMD'

Let's check our dataset:

In [ ]:
# List files in the directory
files = os.listdir(dir_dataset)

# Print the list of files
print(files)

We have **4 audio files (WAV)** and **4 CSV files** with the corresponding pitch annotation.

Let's just memorize the 4 file prefixes:

In [ ]:
file_prefixes = sorted(list(set([_[:-4] for _ in files])))
print(file_prefixes)

Let's listen to the first file...

In [ ]:
fn_wav = os.path.join(dir_dataset, file_prefixes[0] + '.wav')
x, fs = librosa.load(fn_wav)
ipd.display(ipd.Audio(data=x, rate=fs))

**Observation**: This a polyphonic piano recording sampled at 22.05kHz

## Feature Extraction

We need a function to compute the **Constant-Q transform** from all these files. 
In contrast to the pitch tracking example from the last seminar, we only use one frequency bin per semitone, so 12 bins per octave.

We want to cover the piano range between MIDI pitch 21 to 108 (https://newt.phys.unsw.edu.au/jw/notes.html), which covers the entire pitch range of a piano. This results in a minimum frequency of 

In [ ]:
f_min = 440 * 2**((21 - 69)/12)
print(f"f_min =  {f_min}Hz")

and

In [ ]:
n_bins = 108-21+1
print(f"{n_bins} frequency bins")

We furthermore need an array that contains the MIDI pitch values that correspond to each frequency bin in the CQT:

In [ ]:
cqt_midi_pitch = np.arange(21, 109)
print(cqt_midi_pitch)

In [ ]:
def compute_cqt(fn_wav, fs=44100, hop_length=256, bpo=12, fmin=31, nbins = 88):
    """
    Compute the Constant-Q transform (CQT) with librosa
    Args:
        fn_wav (str): File path to the input audio file (in .wav format).
        fs (int, optional): Sampling rate of the audio file (in Hz). Default is 22050 Hz.
        hop_length (int, optional): Number of samples between successive CQT columns. Default is 256.
        bpo (int, optional): Number of bins per octave for the CQT. Default is 12.
        fmin (int, optional): Minimum frequency (in Hz) of the CQT. Default is 31 Hz.
        nbins (int, optional): Number of bins in the CQT. Default is 320.

    Returns:
        cqt (np.ndarray): 2D array containing the magnitude CQT coefficients.
        time_axis (np.ndarray): Numpy array with frame times in seconds.
        freq_hz (np.ndarray): Frequency values in Hz of all frequency bands of the CQT.
    """
    x, fs = librosa.load(fn_wav, sr=fs, mono=True)
    cqt = np.abs(librosa.core.cqt(x, sr=fs, 
                                  hop_length=hop_length, 
                                  fmin=fmin,
                                  bins_per_octave=bpo,
                                  n_bins=nbins))
    cqt = librosa.amplitude_to_db(cqt, ref=np.max)
    cqt = cqt.astype(np.float16)
    time_axis = np.arange(cqt.shape[1])*hop_length/fs
    freq_hz = librosa.cqt_frequencies(n_bins=nbins, fmin=fmin, bins_per_octave=bpo)

    return cqt, time_axis, freq_hz
    

In [ ]:
# let's try it:
cqt, cqt_times_sec, cqt_freqs = compute_cqt(fn_wav)

print(f"Shape of CQT matrix {cqt.shape}")

# plot the first part of the CQT
pl.figure(figsize=(10,3))
librosa.display.specshow(cqt[:, :1000], sr=44100, x_axis='time', y_axis='cqt_note', ax=pl.gca())
pl.show()

**Observation**: We can see that the recording starts with a **monophonic** piano line until around 5s, then it becomes **polyphonic**, i.e., multiple notes are audible at the same time

## Import pitch annotations

In [ ]:
# let's inspect the first CSV file
fn_csv = os.path.join(dir_dataset, file_prefixes[0] + '.csv')
data = np.loadtxt(fn_csv, delimiter=';', skiprows=1, usecols=(0, 1, 2))

onset_sec = data[:, 0]
duration_sec = data[:, 1]
pitch = data[:, 2].astype(int)

# let's check the first three notes
print(onset_sec[:3])
print(duration_sec[:3])
print(pitch[:3])


**Observation**: 

The first column includes **note onset times in seconds**, the second column **the note duration in seconds**, while the third column contains the **pitch encoded as MIDI pitch**.

## Annotation Mapping

**Problem**: 

We have a) our multipitch annotations and b) the time axis of our CQT, which are **not matching**.

**Goal**

We want a binary matrix of the same shape as our **CQT** matrix, which has **ones** wherever a pitched frame is annotated.

In [ ]:
def create_cqt_target(cqt_times_sec, cqt_midi_pitch, note_onset_sec, note_duration_sec, note_pitch):
    
    n_frames = len(cqt_times_sec)
    n_bins = len(cqt_midi_pitch)
    
    cqt_target = np.zeros((n_bins, n_frames), dtype=bool)
    
    n_notes = len(note_pitch)
    print(n_notes)
    for i in range(n_notes):
        
        # find closest frequency bin
        pitch = np.where(cqt_midi_pitch == note_pitch[i])[0]

        start_frame = np.argmin(np.abs(cqt_times_sec-note_onset_sec[i]))
        end_frame = np.argmin(np.abs(cqt_times_sec-(note_onset_sec[i]+note_duration_sec[i])))
        
        # "mark" it as "True"
        cqt_target[pitch, start_frame:end_frame] = True
            
    return cqt_target

In [ ]:
# let's try this again for the first song
fn_wav = os.path.join(dir_dataset, file_prefixes[0] + '.wav')
cqt, cqt_times_sec, cqt_freq_hz = compute_cqt(fn_wav)

# load annotation file
fn_csv = os.path.join(dir_dataset, file_prefixes[0] + '.csv')
data = np.loadtxt(fn_csv, delimiter=';', skiprows=1, usecols=(0, 1, 2))
onset_sec = data[:, 0]
duration_sec = data[:, 1]
pitch = data[:, 2].astype(int)

# create targets for CQT matrix
cqt_target = create_cqt_target(cqt_times_sec, cqt_midi_pitch, onset_sec, duration_sec, pitch)

# finally, let's plot it 
pl.figure(figsize=(15,5))
pl.subplot(1,2,1)
librosa.display.specshow(cqt[:, :1000], sr=44100, x_axis='time', y_axis='cqt_note', ax=pl.gca())
pl.title('CQT')
pl.subplot(1,2,2)
pl.imshow(cqt_target[:, :1000], aspect="auto", origin="lower", cmap="Greys")
pl.title('Target')
pl.tight_layout()
pl.show()


**Observation**

The **piano-roll** target on the right the active pitches over time. This is what we want our model to predict.

## Batch Feature Extraction

Nice, so let's compute all 4 CQT spectrograms and all 4 target matrices: (**this takes some seconds ...**)

In [ ]:
all_cqts = []
all_targets = []

for i in range(4):
    print(f"Process file {i+1}/4")
    # (1) compute CQT
    fn_wav = os.path.join(dir_dataset, file_prefixes[i] + '.wav')
    cqt, cqt_times_sec, cqt_freq_hz = compute_cqt(fn_wav)
    
    # (2) compute target matrix
    fn_csv = os.path.join(dir_dataset, file_prefixes[i] + '.csv')
    data = np.loadtxt(fn_csv, delimiter=';', skiprows=1, usecols=(0, 1, 2))
    onset_sec = data[:, 0]
    duration_sec = data[:, 1]
    pitch = data[:, 2].astype(int)
    cqt_target = create_cqt_target(cqt_times_sec, cqt_midi_pitch, onset_sec, duration_sec, pitch)
    
    all_cqts.append(cqt)
    all_targets.append(cqt_target)

# we'll use the first three files as training and the final file as test set
all_cqts_train = np.hstack(all_cqts[:3])
all_targets_train = np.hstack(all_targets[:3])

all_cqts_test = all_cqts[3]
all_targets_test = all_targets[3]

print(f"Final shape (training) = {all_cqts_train.shape}")
print(f"Final shape (test) = {all_cqts_test.shape}")

    

Let's **visualize** all CQTs and target matrices...

In [ ]:
pl.figure(figsize=(8, 15))
n = len(all_cqts)
for i in range(n):
    pl.subplot(2*n, 1, 2*i+1)
    pl.imshow(all_cqts[i], aspect="auto")
    pl.axis("off")    
    pl.subplot(2*n, 1, 2*i+2)
    pl.imshow(all_targets[i], aspect="auto")
    pl.axis("off")
pl.tight_layout()
pl.show()

## Neural Network

### Patches

For simplicity, well cut our matrices into patches of a fixed size (256 frames) with no overlap.

In [ ]:
feat_train = []
target_train = []
n_frames = all_cqts_train.shape[1]
offset = 0
patch_len = 256
patch_hop = 256

while True:
    if offset + patch_len >= n_frames:
        break
    feat_train.append(all_cqts_train[:, offset: offset + patch_len])
    target_train.append(all_targets_train[:, offset: offset + patch_len])
    offset += patch_hop
feat_train = np.stack(feat_train)
target_train = np.stack(target_train)

# finally, we'll add a singleton dimension (one channel)
feat_train = np.expand_dims(feat_train, -1)
target_train = np.expand_dims(target_train, -1)

print(f"Feature & target array shape = {feat_train.shape}")



## Neural Network Architecture

We're using the **functional API** (https://keras.io/guides/functional_api/) to set up a **simple UNet** model.

In [ ]:
def conv_block(x, filters, kernel_size, padding="same", act="relu"):
    x = BatchNormalization()(x)
    x = Conv2D(filters, kernel_size, padding=padding, activation=act)(x)
    return x

In [ ]:
def create_unet(input_shape):
    # Define input layer
    inputs = Input(shape=input_shape)
    
    n_filters_encoder = [16, 32, 64]
    n_filters_decoder = [64, 32, 1]
    
    # (1) Encoder
    cb_enc_1 = conv_block(inputs, filters=n_filters_encoder[0], kernel_size=(3,3), padding="same")
    mp_enc_1 = MaxPool2D(pool_size=(2, 2))(cb_enc_1)
    
    cb_enc_2 = conv_block(mp_enc_1, filters=n_filters_encoder[0], kernel_size=(3,3), padding="same")
    mp_enc_2 = MaxPool2D(pool_size=(2, 2))(cb_enc_2)
    
    cb_enc_3 = conv_block(mp_enc_2, filters=n_filters_encoder[0], kernel_size=(3,3), padding="same")
    mp_enc_3 = MaxPool2D(pool_size=(2, 2))(cb_enc_3)
    
    # (2) Decoder
    us_dec_1 = UpSampling2D(size=(2, 2))(mp_enc_3)
    us_dec_1 = concatenate([us_dec_1, cb_enc_3])                      
    cb_dec_1 = conv_block(us_dec_1, filters=n_filters_decoder[0], kernel_size=(3,3), padding="same")

    us_dec_2 = UpSampling2D(size=(2, 2))(cb_dec_1)
    us_dec_2 = concatenate([us_dec_2, cb_enc_2])                      
    cb_dec_2 = conv_block(us_dec_2, filters=n_filters_decoder[1], kernel_size=(3,3), padding="same")

    us_dec_3 = UpSampling2D(size=(2, 2))(cb_dec_2)
    print(us_dec_3.shape, cb_enc_1.shape)
    us_dec_3 = concatenate([us_dec_3, cb_enc_1])                      
    cb_dec_3 = conv_block(us_dec_3, filters=n_filters_decoder[2], kernel_size=(3,3), padding="same", act="sigmoid")

    
    
    # Create model
    model = Model(inputs=inputs, outputs=cb_dec_3)
    
    model.compile(loss='binary_crossentropy', 
              optimizer='adam')
 
    return model
    
# Example usage
input_shape = feat_train.shape[1:] 
model = create_unet(input_shape)
model.summary()
    

## Model Training

In [ ]:
history = model.fit(feat_train, target_train, epochs=30, batch_size=4, verbose=1)

In [ ]:
pl.plot(history.history['loss'])
pl.ylabel('Loss')
pl.xlabel('Epoch')
pl.show()

## Model Test

We're testing our model by making predictions on the fourth file.
The approach is to 
 - cut in into similarly sized patches 
 - compute prediction for each patch
 - concatenate the predictions

In [ ]:
print(all_cqts_test.shape)

In [ ]:
all_pred = []
offset = 0
n_frames = all_cqts_test.shape[1]
while True:
    patch = all_cqts_test[:, offset:offset+patch_len]
    patch = np.expand_dims(patch, 0)
    patch = np.expand_dims(patch, -1)
    
    # compute model prediction for current 
    curr_pred = model.predict(patch)
    # reduce to 2D piano roll (pitch vs. time)
    curr_pred = curr_pred[0, :, :, 0]
    all_pred.append(curr_pred)
    
    offset += patch_len
    
    if offset > n_frames-patch_len:
        break
        
# finally, let's combine all patch predictions into one piano roll
all_pred = np.hstack(all_pred)

print(f"Final shape: {all_pred.shape}")

## Result Visualization

In [ ]:
print(all_targets_test.shape)

pl.figure(figsize=(15,6))
pl.subplot(2,2,1)
librosa.display.specshow(all_cqts_test, sr=44100, x_axis='time', y_axis='cqt_note', ax=pl.gca())
pl.title('CQT')
pl.subplot(2,2,2)
pl.imshow(all_pred, aspect="auto", origin="lower",cmap="Greys")
pl.title('Predicted Pitch Contour')
pl.subplot(2,2,4)
pl.imshow(all_targets_test, aspect="auto", origin="lower",cmap="Greys")
pl.title('True Pitch Contour')
pl.tight_layout()
pl.show()

In [ ]:
fn_wav = os.path.join(dir_dataset, file_prefixes[3] + '.wav')
x, fs = librosa.load(fn_wav)
ipd.display(ipd.Audio(data=x, rate=fs))

## Results

That looks promising, still several overtones end up as f0 candidates but this usually is a matter of more training data.


## Possible next steps

- run a proper evaluation by comparing the predicted piano roll with the ground truth using the **mir_eval** toolbox (https://craffel.github.io/mir_eval/#module-mir_eval.multipitch)
...